In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as wdg
import time
from IPython import display
from glob import glob
import cv2

In [2]:
#Busqueda y creacion del directorio del output
def create_dir(path):
    try:
        if not os.path.exists(path):
            os.makedirs(path)
    except OSError:
        print(f"ERROR: creating directory with name {path}")



In [3]:
def calculate_corrcoef(frameBW, frame_ant):
    arr1 = np.asarray(frameBW)           
    arr2 = np.asarray(frame_ant)
    try:
        corrcoef = np.corrcoef(arr1.flat,arr2.flat)[1,0]
    except FloatingPointError:
        print('Error has been raised')
    return corrcoef

In [4]:
def View_Progress(x,Max_length):
        Output = wdg.IntProgress(
        value = x,
        min=0,
        max=Max_length,
        description='Loading:',
        bar_style='', # 'success', 'info', 'warning', 'danger' or ''
        style={'bar_color': 'maroon'},
        orientation='horizontal'
        )
        return Output

In [5]:
def Count_frame(video_path):
    
    cap = cv2.VideoCapture(video_path)
    
    idx = 0
    while True:
        ret, frame = cap.read()
        if idx == 0:
            height, width  = frame.shape[:2]
        if ret == False:
            cap.release()
            break
        idx += 1
    return idx, height, width

In [6]:
def Show_Cantframes(length, length_sum, save_path):
 
    Porcentaje = (100*length_sum)/length
    ## Declaramos valores para el eje x
    eje_x = ['Original :'+str(length), 'Resumen :'+str(length_sum)]

    ## Declaramos valores para el eje y
    eje_y = [length,length_sum]

    ## Creamos Gráfica
    plt.bar(eje_x, eje_y)

    ## Legenda en el eje y
    plt.ylabel('Cantidad de fotogramas')

    ## Legenda en el eje x
    plt.xlabel('video')

    ## Título de Gráfica
    plt.title('Porcentaje de fotogramas resumidos: '+str(Porcentaje))

    ## Mostramos Gráfica
    plt.savefig(f"{save_path}/Diagram.png")
    
    plt.cla()

    print("Porcentaje de fotogramas reducidos: "+str(Porcentaje))

In [7]:

def save_frame(video_path, save_dir, esti_error):
    name = video_path.split("\\")[-1].split(".")[0]
    save_path = os.path.join(save_dir, name)
    create_dir(save_path)
    display.clear_output(wait=True)
    cap = cv2.VideoCapture(video_path)    
    frame_ant = []    
    corrcoef_def = 1 - (esti_error)
    corrcoef = 0
    length, height, width = Count_frame(video_path)
    video = cv2.VideoWriter(f"{save_path}/{name}.mp4", cv2.VideoWriter_fourcc(*'mp4v'), 4, (width,height))
    idx = 0
    length_sum = 0
    while True:
        #Verificar y leer cada frame del video
        ret, frame = cap.read()                
        # Condicional en caso no exista mas frames a leer
        if ret == False:
            cap.release()
            break
        #Conversion de frame de RGB a Escala de grises    
        frameBW = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
       
        #Cantidad de frames que vamos a guardar filtrado por la correlacion de pearson        
        if idx == 0:           
            cv2.imwrite(f"{save_path}/{idx}.png", frame)
            video.write(frame)
            length_sum += 1
        if idx > 0:
            corrcoef = calculate_corrcoef(frameBW, frame_ant)
            #Comparacion entre pares de frames consecutivos
            if corrcoef <= corrcoef_def:           
                cv2.imwrite(f"{save_path}/{idx}.png", frame)
                video.write(frame)
                length_sum += 1
        frame_ant = frameBW 
        Output = View_Progress(idx,length)       
        display.display(Output) 
        display.clear_output(wait=True)
        time.sleep(0.05)   
        idx += 1
        

    video.release()    
    
    Show_Cantframes(idx, length_sum, save_path)

In [8]:
def Capture_inf():
    Input = wdg.BoundedFloatText(
    value=0.5,
    min=0,
    max=1.0,
    step=0.1,
    description='Esti_Error:',
    disabled=False
    )
    button = wdg.Button(description="Resume!",disabled=False)
    output = wdg.Output()
    
    def on_button_clicked(b):
        with output:
            video_paths = glob("Videos/*")
            save_dir = "save"
            error = float(Input.value)
            
            for path in video_paths:
                save_frame(path, save_dir, error)
    
    display.display(Input, button, output)
    button.on_click(on_button_clicked)
    
    


In [9]:
# Introduzca el error y resuma los videos
if __name__== "__main__":
    Capture_inf()
       

BoundedFloatText(value=0.5, description='Esti_Error:', max=1.0, step=0.1)

Button(description='Resume!', style=ButtonStyle())

Output()